# Iteration 1

## Connecting Colabs to BigQuery project

In [ ]:
#  Install Required Packages in Colab
!pip install --upgrade google-cloud-bigquery

In [ ]:
# Authenticate with Google Account
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving cinemaparadiso-462409-f2ba92221c1f.json to cinemaparadiso-462409-f2ba92221c1f.json


In [ ]:
from google.oauth2 import service_account
from google.cloud import bigquery

key_path = "cinemaparadiso-462409-f2ba92221c1f.json"

credentials = service_account.Credentials.from_service_account_file(key_path)

project_id = "cinemaparadiso-462409"

client = bigquery.Client(credentials=credentials, project=project_id)
client = bigquery.Client(project="cinemaparadiso-462409")

## Import files from Big Query


In [ ]:
import pandas as pd

In [ ]:
# Read table cleaned_imdb_movies
query = "SELECT * FROM `cinemaparadiso-462409.cinema_paradiso.fa_list_statistics_imdb_merged_bygenre`"
df_movie_scores = client.query(query).to_dataframe()

## Data organisation

In [ ]:
# TABLE TO COUNT THE NUMBER OF MOVIES PER GENRE

# Check the columns
df_movie_scores.columns

Index(['title', 'year', 'rating', 'votes', 'duration', 'genre',
       'weighted_score', 'genre_action', 'genre_adventure', 'genre_animation',
       'genre_comedy', 'genre_crime', 'genre_documentary', 'genre_drama',
       'genre_family', 'genre_fantasy', 'genre_history', 'genre_horror',
       'genre_musical', 'genre_music', 'genre_mystery', 'genre_romance',
       'genre_scifi', 'genre_tv_movie', 'genre_thriller', 'genre_war',
       'genre_western', 'genre_biography', 'genre_short', 'genre_film_noir'],
      dtype='object')

In [ ]:
# Create a list with the columns that I want to analyse
genres_list = ['genre_action', 'genre_adventure', 'genre_animation',
       'genre_comedy', 'genre_crime', 'genre_documentary', 'genre_drama',
       'genre_family', 'genre_fantasy', 'genre_history', 'genre_horror',
       'genre_musical', 'genre_music', 'genre_mystery', 'genre_romance',
       'genre_scifi','genre_thriller', 'genre_war',
       'genre_western', 'genre_biography', 'genre_film_noir']

In [ ]:
# Do a for loop to access all the genres in the list genres_list.
# In each iteration, gropu by the corresponding genre (only where the value is 1) and calculate the mean of the weighted score.
# In each iretarion, add list with the mean weithed score and as the key we store the genre. We add every iteratio in the created empty dictionary.
dict = {}

for genres in genres_list:
  genre_ratings_score = df_movie_scores.groupby(genres)["weighted_score"].mean().loc[1,]
  dict[genres.replace("genre_","",1)] = [genre_ratings_score]


In [ ]:
# Transform the dictionary into a dataframe with column names
df_movie_scores_by_genre = pd.DataFrame(dict).transpose().reset_index().rename(columns={"index":"genre",0:"weighted_score"})

In [ ]:
# Create a csv file
csv_filename = 'movie_scores_by_genre.csv'
df_movie_scores_by_genre.to_csv(csv_filename, index=False)

In [ ]:
# Order descendent by weighted score
df_movie_scores_by_genre = df_movie_scores_by_genre.sort_values(by="weighted_score", ascending=False)

In [ ]:
# Number of movies per genre: agregate each of the genres to sum the 1
df_number_movies_per_genre = df_movie_scores[genres_list].sum()
df_number_movies_per_genre = df_number_movies_per_genre.reset_index()
df_number_movies_per_genre.columns = ["genre", "number_of_movies"]
df_number_movies_per_genre = df_number_movies_per_genre.sort_values(by="number_of_movies", ascending=False)

In [ ]:
# Create a csv file
csv_filename = 'df_number_movies_per_genre.csv'
df_number_movies_per_genre.to_csv(csv_filename, index=False)

## Data visualisation

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
# Single charts
fig_bar_list_movies = px.bar(df_movie_scores_by_genre,
                            x="genre",
                            y="weighted_score",
                            title="Rate by genre (1203 movies)"
                            )

fig_pie_list_movies = px.pie(df_number_movies_per_genre,
                            values="number_of_movies",
                            names="genre",
                            title="Number of movies per genre",
                            )

# Subplot layout
fig_combined = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Rate by Genre", "Number of Movies per Genre"),
    specs=[[{"type": "xy"}, {"type": "domain"}]]  # bar is xy, pie is domain
)

# Add charts
for trace in fig_bar_list_movies.data:
    fig_combined.add_trace(trace, row=1, col=1)
for trace in fig_pie_list_movies.data:
    fig_combined.add_trace(trace, row=1, col=2)

# Update layout
fig_combined.update_layout(height=500, width=1000, title_text="List_movies")

# Show figure
fig_combined.show()

# Conclusions
>> 1. Most represented = most popular movies not the same as most rated.
>> 2. Most rated coincides to less movies present -> niche?

>> Comments: It seems like we should have into account also how many movies per genre we have. (I doubt film-noir is the best scored genre, even it is great). I think we are giving better scores to less represented movies.
There are good reviewd movies that are underserved.

# Next steps
>> 1. Top 10 genres, languages, directors, and actors by number of
films: popular
>> 2. Top 10 genres, languages, etc. by average weighted score: good (should I add a threshold?) Undersereved, good review, but low number of movies.
>> 3. Clustering of movies?